In [10]:
%pip install pdf2image pytesseract

In [11]:
# Install poppler-utils
!apt-get update
!apt-get install -y poppler-utils

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [12]:
# Install Tesseract OCR and Bengali language pack
!apt-get update
!apt-get install -y tesseract-ocr
!apt-get install -y tesseract-ocr-ben

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [21]:
%pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 33.7 MB/s eta 0:00:00


Preprocessing and Cleaned text generation

In [19]:
import os
from pdf2image import convert_from_path
import pytesseract
import re
import unicodedata

# Configured pytesseract to point to the Tesseract executable in Colab
pytesseract.pytesseract.tesseract_cmd = r"/usr/bin/tesseract"

def extract_text_from_pdf_via_ocr(pdf_path: str, lang: str = 'ben') -> str:
    try:
        print("Converting PDF to images...")
        if not os.path.exists(pdf_path):
            print(f"Error: PDF file not found at {pdf_path}")
            return ""

        images = convert_from_path(pdf_path, dpi=300)
        print(f"Total pages: {len(images)}")
    except Exception as e:
        print(f"Error converting PDF to images: {e}")
        return ""

    full_text = ""
    for i, img in enumerate(images):
        print(f"Processing page {i + 1}...")
        try:
            text = pytesseract.image_to_string(img, lang=lang)
        except Exception as e:
            print(f"Error processing page {i + 1}: {e}")
            text = ""
        full_text += f"\n\n--- Page {i + 1} ---\n{text}"

    return full_text

def clean_bangla_text(raw_text: str) -> str:
    """Clean and normalized Bangla OCR text."""
    print("Cleaning extracted text...")

    # Normalizing Unicode characters
    text = unicodedata.normalize("NFKC", raw_text)

    # Removing extra whitespace and line breaks
    text = re.sub(r'[ \t]+', ' ', text)
    text = re.sub(r'\n{2,}', '\n', text)

    # Removing page markers
    text = re.sub(r'--- Page \d+ ---', '', text)

    # Removing non-Bangla artifacts (keep Bangla, whitespace, punctuation)
    text = re.sub(r'[^\u0980-\u09FF\s.,?!\-\(\)\’\"“”]', '', text)

    return text.strip()

def save_text(text: str, output_path: str):
    """Saved cleaned text to a .txt file."""
    try:
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(text)
        print(f"Cleaned Bangla text saved to: {output_path}")
    except Exception as e:
            print(f"Error saving text: {e}")

if __name__ == "__main__":
    pdf_path = r"/content/HSC26-Bangla1st-Paper.pdf"
    output_dir = "/content/Cleaned_text"
    output_filename = "HSC26-Bangla1st-Paper.txt"
    output_path = os.path.join(output_dir, output_filename)


    ocr_raw = extract_text_from_pdf_via_ocr(pdf_path, lang="ben")
    if ocr_raw:
        cleaned_text = clean_bangla_text(ocr_raw)
        save_text(cleaned_text, output_path)
    else:
        print("No text extracted from the PDF.")

Converting PDF to images...
Total pages: 49
Processing page 1...
Processing page 2...
Processing page 3...
Processing page 4...
Processing page 5...
Processing page 6...
Processing page 7...
Processing page 8...
Processing page 9...
Processing page 10...
Processing page 11...
Processing page 12...
Processing page 13...
Processing page 14...
Processing page 15...
Processing page 16...
Processing page 17...
Processing page 18...
Processing page 19...
Processing page 20...
Processing page 21...
Processing page 22...
Processing page 23...
Processing page 24...
Processing page 25...
Processing page 26...
Processing page 27...
Processing page 28...
Processing page 29...
Processing page 30...
Processing page 31...
Processing page 32...
Processing page 33...
Processing page 34...
Processing page 35...
Processing page 36...
Processing page 37...
Processing page 38...
Processing page 39...
Processing page 40...
Processing page 41...
Processing page 42...
Processing page 43...
Processing page 44.

Chunking and Vectorization

In [23]:
import os
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
import faiss

# Update the input file path to where the cleaned text was saved
INPUT_TEXT_FILE = "/content/Cleaned_text/HSC26-Bangla1st-Paper.txt"

CHUNK_SIZE = 500
CHUNK_OVERLAP = 50
EMBEDDING_MODEL = "distiluse-base-multilingual-cased-v1"

FAISS_INDEX_PATH = "vector_store/faiss_index.bin"
CHUNKS_PATH = "vector_store/chunks.pkl"

def load_clean_text(path: str) -> str:
    """Load cleaned text from a .txt file."""
    try:
        with open(path, "r", encoding="utf-8") as f:
            return f.read()
    except FileNotFoundError:
        print(f"Error: Input file not found at {path}")
        return ""
    except Exception as e:
        print(f"Error loading text file: {e}")
        return ""


def split_text_into_chunks(text: str, chunk_size=500, overlap=50) -> list:
    """Splited text into overlapping chunks."""
    print("Splitting text into chunks...")
    if not text:
        print("No text to split.")
        return []
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
    chunks = splitter.split_text(text)
    print(f"Total chunks created: {len(chunks)}")
    return chunks

# ----------------------------
# STEP 3: Generate embeddings
# ----------------------------

def embed_chunks(chunks: list, model_name: str):
    """Generate embeddings for text chunks using a Sentence Transformer model."""
    if not chunks:
        print("No chunks to embed.")
        return np.array([]), None

    print(f"Loading embedding model: {model_name}")
    try:
        model = SentenceTransformer(model_name)
    except Exception as e:
        print(f"Error loading embedding model: {e}")
        return np.array([]), None

    print("Generating embeddings...")
    embeddings = model.encode(chunks, show_progress_bar=True)
    return embeddings, model

# ----------------------------
# STEP 4: Save to FAISS
# ----------------------------

def save_to_faiss(embeddings: np.ndarray, chunks: list, faiss_path: str, chunks_path: str):
    """Save embeddings to FAISS index and chunks to a pickle file."""
    if embeddings.size == 0:
        print("No embeddings to save.")
        return

    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings))

    try:
        os.makedirs(os.path.dirname(faiss_path) or '.', exist_ok=True) # Create dir for faiss index if path has one
        faiss.write_index(index, faiss_path)
    except Exception as e:
        print(f"Error saving FAISS index: {e}")
        return

    try:
        os.makedirs(os.path.dirname(chunks_path) or '.', exist_ok=True) # Create dir for chunks if path has one
        with open(chunks_path, "wb") as f:
            pickle.dump(chunks, f)
    except Exception as e:
        print(f"Error saving chunks: {e}")
        return

    print(f"FAISS index saved to: {faiss_path}")
    print(f"Chunks saved to: {chunks_path}")

# ----------------------------
# MAIN PIPELINE
# ----------------------------

if __name__ == "__main__":
    os.makedirs("vector_store", exist_ok=True)

    text = load_clean_text(INPUT_TEXT_FILE)
    if text:
        chunks = split_text_into_chunks(text, CHUNK_SIZE, CHUNK_OVERLAP)
        if chunks:
            embeddings, _ = embed_chunks(chunks, EMBEDDING_MODEL)
            if embeddings.size > 0:
                save_to_faiss(np.array(embeddings), chunks, FAISS_INDEX_PATH, CHUNKS_PATH)
            else:
                print("Failed to generate embeddings.")
        else:
            print("Failed to split text into chunks.")
    else:
        print("Failed to load clean text.")

Splitting text into chunks...
Total chunks created: 202
Loading embedding model: distiluse-base-multilingual-cased-v1


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

2_Dense/pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

Generating embeddings...


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

FAISS index saved to: vector_store/faiss_index.bin
Chunks saved to: vector_store/chunks.pkl


Vector Database

In [53]:
import os
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import google.generativeai as genai  # Gemini library
from google.colab import userdata
from collections import deque

# ----------------------------
# Memory Setup (Short-Term)
# ----------------------------
chat_memory = deque(maxlen=5)  # Store last 5 query-answer pairs

# ----------------------------
# API Key Setup
# ----------------------------
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    print("Google API key not found. Please set it in Colab secrets (name it 'GOOGLE_API_KEY').")
    exit()

genai.configure(api_key=GOOGLE_API_KEY)

# ----------------------------
# Configuration
# ----------------------------
EMBEDDING_MODEL = "distiluse-base-multilingual-cased-v1"
FAISS_INDEX_PATH = "vector_store/faiss_index.bin"
CHUNKS_PATH = "vector_store/chunks.pkl"
TOP_K = 3  # How many chunks to retrieve
GEMINI_MODEL = genai.GenerativeModel("gemini-1.5-flash-latest")

# ----------------------------
# Load Vector DB and Chunks
# ----------------------------

def load_faiss_index(index_path: str):
    try:
        index = faiss.read_index(index_path)
        print("FAISS index loaded.")
        return index
    except Exception as e:
        print(f"Error loading FAISS index: {e}")
        return None

def load_chunks(chunks_path: str):
    try:
        with open(chunks_path, "rb") as f:
            chunks = pickle.load(f)
        print("Chunks loaded.")
        return chunks
    except Exception as e:
        print(f"Error loading chunks: {e}")
        return []

# ----------------------------
# Embedding & Retrieval
# ----------------------------

def embed_query(query: str, model: SentenceTransformer) -> np.ndarray:
    return model.encode([query])

def retrieve_top_chunks(query: str, model: SentenceTransformer, index, chunks: list, top_k=3) -> list:
    print("Embedding query and retrieving top chunks...")
    query_vec = embed_query(query, model)
    distances, indices = index.search(np.array(query_vec), top_k)
    return [chunks[i] for i in indices[0]]

# ----------------------------
# MAIN Interactive RAG Loop
# ----------------------------

if __name__ == "__main__":
    model = SentenceTransformer(EMBEDDING_MODEL)
    index = load_faiss_index(FAISS_INDEX_PATH)
    chunks = load_chunks(CHUNKS_PATH)

    if index and chunks:
        print("\n Ask me anything in English or Bangla (type 'exit' to quit):\n")

        while True:
            query = input("Your Question: ").strip()
            if query.lower() in ['exit', 'quit']:
                print("Exiting.")
                break

            # Step 1: Retrieve context chunks
            top_chunks = retrieve_top_chunks(query, model, index, chunks, TOP_K)

            # Step 2: Prepare short-term memory
            memory_context = "\n".join(f"Q: {q}\nA: {a}" for q, a in chat_memory)

            # Step 3: Format retrieved document context
            context_str = ''.join(f'- {chunk.strip()}\n' for chunk in top_chunks)

            # Step 4: Combine everything into a prompt
            full_prompt = f"""Use the following memory and context to answer the question in Bangla or English.

Short-term memory:
{memory_context}

Retrieved document context:
{context_str}

Question: {query}

Answer:"""

            # Step 5: Generate the answer from Gemini
            try:
                response = GEMINI_MODEL.generate_content(full_prompt)
                answer = response.text.strip()
            except Exception as e:
                print(f"Error generating answer: {e}")
                answer = "Could not generate an answer at this time."

            # Step 6: Store in short-term memory and show result
            chat_memory.append((query, answer))
            print("\n Answer:\n" + answer + "\n")

    else:
        print(" Setup incomplete. Cannot run RAG pipeline.")


✅ FAISS index loaded.
✅ Chunks loaded.

💬 Ask me anything in English or Bangla (type 'exit' to quit):

🧠 Your Question: অনুপেমর ভাষায় সুপুরুষ কােক বলা হেয়েছ?
🔎 Embedding query and retrieving top chunks...

📝 Answer:
প্রশ্নটিতে উল্লেখিত তথ্য অনুসারে উত্তর দেওয়া সম্ভব নয়।  উপস্থাপিত  পাঠ্যে অনুপেমের ভাষায় সুপুরুষ কাকে বলা হয় সে সম্পর্কে কোন তথ্য নেই।



KeyboardInterrupt: Interrupted by user